https://learnche.org/4G3/Mixed-Integer_linear_programming
https://docs.google.com/document/d/1lmBL1Bh251VYb7NmBD2ER-JPD2DSExGR87nu7AWs7oo/edit
https://docs.google.com/document/d/1VZRQ_aaQjMiK_brME9y1dhHYtkhTI_2NLt4PQ51OyhM/edit

In [227]:
from pyomo.environ import *
import pandas as pd
model = ConcreteModel()

In [228]:
model.j = RangeSet( 3 )
model.jp= RangeSet( 3 )
model.k=RangeSet(2) #Units available
#p={1:15,2:6,3:9}
index_p=pd.MultiIndex.from_product([model.j,model.k])
p=pd.DataFrame({'Pij':[15,7,6,14,9,3]})
M=p.sum(axis=0)
p=p.set_index(index_p)
#model.p=Param(model.j,initialize=p)
model.x=Var(model.j,model.k,within=NonNegativeReals)
model.y=Var(model.j,model.jp,model.k,within=Binary)


In [229]:
print(p['Pij'])

j  k
1  1    15
   2     7
2  1     6
   2    14
3  1     9
   2     3
Name: Pij, dtype: int64


In [230]:
M

Pij    54
dtype: int64

In [231]:
model.Obj = Objective(expr=(sum(sum(model.x[j,k]+p['Pij'][j,k] for j in model.j) for k in model.k)) , sense=minimize)

In [232]:
def disj1 (model,j,jp,k):
    if j<jp:
        return model.x[j,k] + p['Pij'][j,k]  <= model.x[jp,k] + M['Pij']*(1-model.y[j,jp,k])
    else:
        return model.x[j,k] >=0
def disj2 (model,j,jp,k):
    if j<jp:
        return model.x[jp,k] + p['Pij'][jp,k] <= model.x[j,k] + M['Pij']*model.y[j,jp,k]
    else:
        return model.x[j,k] >=0
def sumy (model,j):
    return sum(model.y[j,jp,k] for k in model.k)==1
model.c1= Constraint( model.j,model.jp,model.k, rule=disj1 )
model.c2= Constraint( model.j,model.jp, model.k, rule=disj2 )

results = SolverFactory('glpk').solve(model).write()
model.pprint()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 88.0
  Upper bound: 88.0
  Number of objectives: 1
  Number of constraints: 37
  Number of variables: 13
  Number of nonzeros: 61
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 37
      Number of created subproblems: 37
  Error rc: 0
  Time: 0.02495741844177246
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------

In [233]:
### Adding a release date
r={1:1,2:1,3:15}
model.r=Param(model.j,initialize=r)
def release(model,j,k):
    return model.x[j,k]>=model.r[j]
model.c3=Constraint(model.j,model.k,rule=release)
results = SolverFactory('glpk').solve(model).write()
model.pprint()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 115.0
  Upper bound: 115.0
  Number of objectives: 1
  Number of constraints: 43
  Number of variables: 13
  Number of nonzeros: 67
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 17
      Number of created subproblems: 17
  Error rc: 0
  Time: 0.024956703186035156
# ----------------------------------------------------------
#   Solution Information
# -------------------------------

In [234]:
### Adding a due date for jobs
d={1:20,2:50,3:50}
model.d=Param(model.j,initialize=d)
def due(model,j,k):
    return model.x[j,k]+p['Pij'][j,k]  <=model.d[j]
model.c4=Constraint(model.j,model.k,rule=due)
results = SolverFactory('glpk').solve(model).write()
model.pprint()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 124.0
  Upper bound: 124.0
  Number of objectives: 1
  Number of constraints: 49
  Number of variables: 13
  Number of nonzeros: 73
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 5
      Number of created subproblems: 5
  Error rc: 0
  Time: 0.0279238224029541
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------